In [1]:
import iql

In [2]:
from StringIO import StringIO  # got moved to io in python3.
import pandas as pd
import requests
sheet = 'https://docs.google.com/spreadsheet/ccc?key=1juJLiXF3R6fj99XA-dfChijbC2AickNAGFA1OBPmBqQ&output=csv'
r = requests.get(sheet)
data = r.content

In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('automatic-docs-7f192b063967.json', scope)
client = gspread.authorize(creds)

#client.openall()
#client.open_by_url('https://docs.google.com/spreadsheets/d/1juJLiXF3R6fj99XA-dfChijbC2AickNAGFA1OBPmBqQ/edit')
#client.del_spreadsheet('1agX17J57HYDwtdSQOXeAlk9rQqJOBuCMOsLjul35Jk4')
wks = client.open_by_key('1juJLiXF3R6fj99XA-dfChijbC2AickNAGFA1OBPmBqQ')

# Extract and print all of the values
#list_of_hashes = sheet.get_all_records()
#print(list_of_hashes)

In [4]:
df = pd.DataFrame(wks.sheet1.get_all_records())

In [5]:
wks.sheet1, wks.worksheet('US')

(<Worksheet 'AU' id:oxg13kv>, <Worksheet 'US' id:oqtxhqd>)

In [48]:
sheets = ['WW', 'US', 'UK', 'CA', 'DE', 'FR', 'AU', 'JP', 'NL', 'IN', 'BR', 'ROW']

In [62]:
data = {}

In [65]:
for i in sheets:
    data[i] = pd.DataFrame(wks.worksheet(i).get_all_records())

In [86]:
def test(df=data['AU']):
    tmp = df.set_index(u'')
    tmp = tmp.ix[1:]
    tmp = tmp.transpose()
    return tmp

In [89]:
#test(data['AU'])

In [101]:
budget_wks = client.open_by_key('1AAZ1HDlGzbhBBchwmwyLC-pC66o3NXBhcXWhG9s8P1Y')

In [102]:
budget = pd.DataFrame(budget_wks.worksheet('2017_Goal_Monthly_Budget').get_all_records())

In [105]:
budget = budget.set_index('Country')

In [123]:
budget['updated_annual_budget'] = budget['updated_annual_budget'].apply(lambda x: x.replace(',','').replace('$',''))
budget['updated_annual_budget'] = budget['updated_annual_budget'].astype(int)

In [124]:
budget['updated_annual_budget'].to_dict()

{'': 134361632,
 'AE': 757622,
 'AR': 144942,
 'AT': 568331,
 'AU': 9518183,
 'BE': 1118991,
 'BH': 13667,
 'BR': 2794344,
 'CA': 2371909,
 'CH': 1925738,
 'CL': 207964,
 'CN': 13,
 'CO': 160640,
 'CR': 15082,
 'CZ': 263524,
 'DE': 15322810,
 'DK': 151168,
 'EC': 19352,
 'EG': 38847,
 'ES': 2307085,
 'FI': 36423,
 'FR': 5054700,
 'GR': 10273,
 'HK': 1353161,
 'HU': 238088,
 'ID': 1217185,
 'IE': 642770,
 'IL': 33292,
 'IN': 9667936,
 'IT': 3077605,
 'JP': 17442207,
 'KR': 320482,
 'KW': 15047,
 'LU': 52093,
 'MA': 24507,
 'MX': 2946933,
 'MY': 420173,
 'NG': 23165,
 'NL': 3890915,
 'NO': 158773,
 'NZ': 1293103,
 'OM': 18054,
 'PA': 11923,
 'PE': 180048,
 'PH': 219575,
 'PK': 49765,
 'PL': 898185,
 'PT': 184764,
 'QA': 57585,
 'RO': 89077,
 'RU': 2300111,
 'SA': 40023,
 'SE': 1862083,
 'SG': 1744935,
 'TH': 96937,
 'TR': 464549,
 'TW': 476362,
 'UA': 47174,
 'UK': 7151747,
 'US': 31857789,
 'UY': 2507,
 'VE': 6481,
 'VN': 243770,
 'ZA': 739147}

In [116]:
import mysqlconnect
def get_budget(year):
    qry = ("SELECT country, SUM(budget) as Budget, SUM(traffic) as Traffic FROM t_budget_traffic "
           "WHERE `last_update`>='{0}-01-01' and `last_update`<='{0}-12-31' and "
           "approval_status_marketing='APPROVED' and approval_status_international='APPROVED' and "
           "approval_status_finance='APPROVED' GROUP BY country;".format(year))
    goals = mysqlconnect.fetch_data(qry, df=True)
    goals = goals[['country', 'Budget']]
    goals['Budget'] = goals['Budget'].astype(int)
    goals = goals.set_index('country')['Budget'].to_dict()
    final = {}
    for i in goals:
        final[i] = goals[i]
    return final

In [117]:
get_budget(2017)

{u'AE': 750000,
 u'AR': 70068,
 u'AT': 534421,
 u'AU': 10000000,
 u'BE': 1100000,
 u'BH': 13172,
 u'BR': 2100000,
 u'CA': 2200000,
 u'CH': 2000000,
 u'CL': 166310,
 u'CN': 0,
 u'CO': 0,
 u'CR': 0,
 u'CZ': 281792,
 u'DE': 15000000,
 u'DK': 129980,
 u'EC': 0,
 u'EG': 45205,
 u'ES': 2390000,
 u'FI': 26000,
 u'FR': 4270000,
 u'GR': 0,
 u'HK': 1500000,
 u'HU': 205791,
 u'ID': 1125000,
 u'IE': 650922,
 u'IL': 0,
 u'IN': 8640000,
 u'IT': 3100000,
 u'JP': 17560000,
 u'KR': 300615,
 u'KW': 0,
 u'LU': 50687,
 u'MA': 0,
 u'MX': 2800000,
 u'MY': 422980,
 u'NG': 0,
 u'NL': 4250000,
 u'NO': 132666,
 u'NZ': 1250000,
 u'OM': 0,
 u'PA': 0,
 u'PE': 72873,
 u'PH': 134309,
 u'PK': 0,
 u'PL': 1000000,
 u'PT': 116292,
 u'QA': 57721,
 u'RO': 0,
 u'RU': 1620000,
 u'SA': 0,
 u'SE': 1950000,
 u'SG': 1800000,
 u'TH': 15285,
 u'TR': 300273,
 u'TW': 463520,
 u'UA': 42809,
 u'UK': 7000000,
 u'US': 32220000,
 u'UY': 0,
 u'VE': 0,
 u'VN': 244971,
 u'ZA': 830000}

In [125]:
import cPickle as pickle

In [126]:
def dict_to_pickle(dict_obj, file_name):
    with open(file_name, 'wb') as pickle_file:
        pickle.dump(dict_obj, pickle_file)


def read_pickle(file_name):
    try:
        with open(file_name, 'rb') as pickle_file:
            return pickle.load(pickle_file)
    except:
        return {}

In [127]:
def run_cache(expiry_time=90):
    edit_time = os.stat('.cache.pickle').st_mtime
    age = time.time() - edit_time
    if age > expiry_time:
        cache_data()